In [4]:
# Imports principais
import sys
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime

# Configuracoes de visualizacao
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("="*60)
print("Ambiente configurado com sucesso!")
print("="*60)
print(f"\nPython: {sys.executable}")
print(f"Requests: {requests.__version__}")
print(f"Pandas: {pd.__version__}")
print(f"Numpy: {np.__version__}")
print(f"Matplotlib: {plt.matplotlib.__version__}")
print(f"Seaborn: {sns.__version__}")
print("\n" + "="*60)


Ambiente configurado com sucesso!

Python: /Volumes/promethion/cath-mainly-alpha/.venv/bin/python
Requests: 2.32.5
Pandas: 2.3.3
Numpy: 2.3.5
Matplotlib: 3.10.7
Seaborn: 0.13.2



In [ ]:
#!/usr/bin/env python3
"""
Script otimizado para ambiente local: baixar estruturas mainly-alpha do CATH
Versao paralela otimizada para macOS
"""

# Configuracoes para ambiente local
BASE_PATH = "/Volumes/promethion/cath-mainly-alpha"
CATH_DOMAIN_LIST_URL = "http://download.cathdb.info/cath/releases/latest-release/cath-classification-data/cath-domain-list.txt"
PDB_DOWNLOAD_URL = "https://files.rcsb.org/download/{}.pdb"
MAINLY_ALPHA_CLASS = "1"  # Class 1 no CATH = Mainly Alpha

# Parametros de otimizacao para processamento paralelo
import multiprocessing
MAX_WORKERS = min(20, multiprocessing.cpu_count() * 2)  # Otimizado para downloads paralelos
CHUNK_SIZE = 1024 * 1024  # 1MB para streaming
REQUEST_TIMEOUT = 30  # Timeout em segundos

def setup_directories():
    """
    Cria estrutura de diretorios local
    """
    base_path = Path(BASE_PATH)
    structures_path = base_path / "structures"
    logs_path = base_path / "logs"

    # Cria diretorios
    base_path.mkdir(parents=True, exist_ok=True)
    structures_path.mkdir(parents=True, exist_ok=True)
    logs_path.mkdir(parents=True, exist_ok=True)

    # Verifica criacao
    print("Verificando estrutura de diretorios:")
    print(f"  Base: {base_path} - {'OK' if base_path.exists() else 'FALHA'}")
    print(f"  Structures: {structures_path} - {'OK' if structures_path.exists() else 'FALHA'}")
    print(f"  Logs: {logs_path} - {'OK' if logs_path.exists() else 'FALHA'}\n")

    if not (base_path.exists() and structures_path.exists() and logs_path.exists()):
        raise RuntimeError("Falha ao criar estrutura de diretorios")

    return base_path, structures_path, logs_path

def download_cath_domain_list(base_path):
    """
    Baixa o arquivo de lista de dominios CATH
    """
    print("Baixando lista de dominios CATH...")

    cath_file = base_path / "cath-domain-list.txt"

    # Nao baixa novamente se ja existe
    if cath_file.exists():
        print(f"Arquivo CATH ja existe: {cath_file}")
        return str(cath_file)

    response = requests.get(CATH_DOMAIN_LIST_URL, timeout=60)
    response.raise_for_status()

    with open(cath_file, 'w') as f:
        f.write(response.text)

    print(f"Arquivo CATH salvo: {cath_file}")
    return str(cath_file)

def parse_cath_domains(cath_file):
    """
    Parse do arquivo CATH para extrair codigos PDB de estruturas mainly-alpha
    """
    pdb_codes = set()

    print("Processando arquivo CATH...")

    with open(cath_file, 'r') as f:
        for line in f:
            if line.startswith('#'):
                continue

            parts = line.strip().split()
            if len(parts) < 2:
                continue

            domain_id = parts[0]
            cath_class = parts[1]

            if cath_class == MAINLY_ALPHA_CLASS:
                pdb_code = domain_id[:4].lower()
                pdb_codes.add(pdb_code)

    print(f"Total de estruturas PDB mainly-alpha: {len(pdb_codes)}")
    return pdb_codes

def download_single_pdb(pdb_code, output_dir):
    """
    Baixa uma unica estrutura PDB
    """
    output_file = output_dir / f"{pdb_code}.pdb"

    # Verifica se ja existe
    if output_file.exists():
        return {
            'pdb_code': pdb_code,
            'status': 'exists',
            'message': 'Arquivo ja existe'
        }

    try:
        url = PDB_DOWNLOAD_URL.format(pdb_code.upper())
        response = requests.get(url, timeout=REQUEST_TIMEOUT, stream=True)
        response.raise_for_status()

        # Download com streaming para economizar memoria
        with open(output_file, 'wb') as f:
            for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
                if chunk:
                    f.write(chunk)

        return {
            'pdb_code': pdb_code,
            'status': 'success',
            'message': 'Download concluido'
        }

    except requests.exceptions.HTTPError as e:
        return {
            'pdb_code': pdb_code,
            'status': 'error',
            'message': f'HTTP {e.response.status_code}'
        }

    except Exception as e:
        return {
            'pdb_code': pdb_code,
            'status': 'error',
            'message': str(e)
        }

def download_structures_parallel(pdb_codes, output_dir, max_workers=MAX_WORKERS):
    """
    Baixa estruturas em paralelo com barra de progresso
    """
    from tqdm.auto import tqdm
    
    pdb_list = sorted(list(pdb_codes))
    total = len(pdb_list)

    results = {
        'total': total,
        'success': 0,
        'exists': 0,
        'error': 0,
        'failed_codes': []
    }

    print(f"\nIniciando download de {total} estruturas...")
    print(f"Workers paralelos: {max_workers}")
    print(f"Salvando em: {output_dir}\n")

    # Download paralelo com barra de progresso
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(download_single_pdb, pdb_code, output_dir): pdb_code
            for pdb_code in pdb_list
        }

        # Barra de progresso
        with tqdm(total=total, desc="Downloading PDB structures") as pbar:
            for future in as_completed(futures):
                result = future.result()

                if result['status'] == 'success':
                    results['success'] += 1
                elif result['status'] == 'exists':
                    results['exists'] += 1
                elif result['status'] == 'error':
                    results['error'] += 1
                    results['failed_codes'].append(result['pdb_code'])

                pbar.update(1)
                pbar.set_postfix({
                    'Success': results['success'],
                    'Exists': results['exists'],
                    'Error': results['error']
                })

    return results

def save_download_results(base_path, pdb_codes, results):
    """
    Salva listas de codigos PDB e resultados
    """
    logs_path = base_path / "logs"
    logs_path.mkdir(parents=True, exist_ok=True)

    if not logs_path.exists():
        raise RuntimeError(f"Falha ao criar diretorio de logs: {logs_path}")

    # Lista completa de codigos
    codes_file = base_path / "mainly_alpha_pdb_codes.txt"
    try:
        with open(codes_file, 'w') as f:
            for code in sorted(pdb_codes):
                f.write(f"{code}\n")
        print(f"Lista de codigos PDB salva em: {codes_file}")
    except Exception as e:
        print(f"Erro ao salvar lista de codigos: {e}")

    # Codigos que falharam
    if results['failed_codes']:
        failed_file = logs_path / "failed_downloads.txt"
        try:
            with open(failed_file, 'w') as f:
                for code in sorted(results['failed_codes']):
                    f.write(f"{code}\n")
            print(f"Codigos com falha salvos em: {failed_file}")
        except Exception as e:
            print(f"Erro ao salvar lista de falhas: {e}")

    # Relatorio detalhado
    report_file = logs_path / "download_report.txt"
    try:
        with open(report_file, 'w') as f:
            f.write("="*60 + "\n")
            f.write("RELATORIO DE DOWNLOAD - CATH MAINLY-ALPHA\n")
            f.write("="*60 + "\n\n")
            f.write(f"Total de estruturas: {results['total']}\n")
            f.write(f"Downloads bem-sucedidos: {results['success']}\n")
            f.write(f"Ja existentes: {results['exists']}\n")
            f.write(f"Falhas: {results['error']}\n")
            f.write(f"Taxa de sucesso: {(results['success'] + results['exists'])/results['total']*100:.1f}%\n")
            f.write("\n" + "="*60 + "\n")
        print(f"Relatorio salvo em: {report_file}")
    except Exception as e:
        print(f"Erro ao salvar relatorio: {e}")

def print_download_report(results, structures_path):
    """
    Exibe relatorio final
    """
    print("\n" + "="*60)
    print("RELATORIO FINAL DE DOWNLOAD")
    print("="*60)
    print(f"Total de estruturas: {results['total']}")
    print(f"Downloads bem-sucedidos: {results['success']}")
    print(f"Ja existentes (nao baixados): {results['exists']}")
    print(f"Falhas: {results['error']}")
    print(f"Taxa de sucesso: {(results['success'] + results['exists'])/results['total']*100:.1f}%")
    print(f"\nEstruturas salvas em: {structures_path}")
    print("="*60)

# ========== EXECUCAO ==========
print("="*60)
print("CATH MAINLY-ALPHA STRUCTURE DOWNLOADER")
print("Ambiente Local - Processamento Paralelo Otimizado")
print("="*60 + "\n")

try:
    # 1. Setup de diretorios
    base_path, structures_path, logs_path = setup_directories()
    print(f"Diretorio base: {base_path}\n")

    # 2. Baixa arquivo CATH
    cath_file = download_cath_domain_list(base_path)

    # 3. Extrai codigos PDB
    pdb_codes = parse_cath_domains(cath_file)

    # 4. Baixa estruturas em paralelo
    results = download_structures_parallel(pdb_codes, structures_path)

    # 5. Salva resultados
    save_download_results(base_path, pdb_codes, results)

    # 6. Relatorio final
    print_download_report(results, structures_path)

except Exception as e:
    print(f"\nErro critico na execucao: {e}")
    import traceback
    traceback.print_exc()


CATH MAINLY-ALPHA STRUCTURE DOWNLOADER
Ambiente Local - Processamento Paralelo Otimizado

Verificando estrutura de diretorios:
  Base: /Volumes/promethion/cath-mainly-alpha - OK
  Structures: /Volumes/promethion/cath-mainly-alpha/structures - OK
  Logs: /Volumes/promethion/cath-mainly-alpha/logs - OK

Diretorio base: /Volumes/promethion/cath-mainly-alpha

Baixando lista de dominios CATH...
Arquivo CATH salvo: /Volumes/promethion/cath-mainly-alpha/cath-domain-list.txt
Processando arquivo CATH...
Arquivo CATH salvo: /Volumes/promethion/cath-mainly-alpha/cath-domain-list.txt
Processando arquivo CATH...
Total de estruturas PDB mainly-alpha: 50996

Iniciando download de 50996 estruturas...
Workers paralelos: 16
Salvando em: /Volumes/promethion/cath-mainly-alpha/structures

Total de estruturas PDB mainly-alpha: 50996

Iniciando download de 50996 estruturas...
Workers paralelos: 16
Salvando em: /Volumes/promethion/cath-mainly-alpha/structures



/Volumes/promethion/cath-mainly-alpha/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

In [ ]:
#!/usr/bin/env python3
"""
Script para limpeza de estruturas PDB em ambiente local
Remove heteroátomos, água, íons e mantém apenas cadeia A
Versão com processamento paralelo otimizado
"""

from pathlib import Path
from typing import Dict
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm
import multiprocessing

# Dependências necessárias (instalar se não tiver):
# pip install biopython tqdm

from Bio import PDB
from Bio.PDB import PDBIO, Select

# Configurações para ambiente local
BASE_PATH = "/Volumes/promethion/cath-mainly-alpha"
MAX_WORKERS = max(1, multiprocessing.cpu_count() - 1)  # Usa CPU-1 para deixar sistema responsivo

# Aminoácidos padrão
STANDARD_AMINO_ACIDS = {
    'ALA', 'ARG', 'ASN', 'ASP', 'CYS',
    'GLN', 'GLU', 'GLY', 'HIS', 'ILE',
    'LEU', 'LYS', 'MET', 'PHE', 'PRO',
    'SER', 'THR', 'TRP', 'TYR', 'VAL'
}

class ChainAOnlySelect(Select):
    """
    Seleciona apenas cadeia A e remove heteroátomos
    """
    def accept_model(self, model):
        return model.id == 0

    def accept_chain(self, chain):
        return chain.id == 'A'

    def accept_residue(self, residue):
        hetfield = residue.id[0]
        if hetfield != ' ':
            return False
        return residue.resname.strip() in STANDARD_AMINO_ACIDS

    def accept_atom(self, atom):
        return True

def setup_directories():
    """
    Verifica e cria diretórios necessários
    """
    base_path = Path(BASE_PATH)

    # Procura estruturas baixadas
    structures_raw = base_path / "structures"
    if not structures_raw.exists():
        print(f"ERRO: Pasta de estruturas não encontrada em {structures_raw}")
        print("Execute primeiro o script de download!")
        return None, None, None, None

    # Cria pastas de saída
    clean_path = base_path / "structures_clean"
    logs_path = base_path / "logs"

    clean_path.mkdir(exist_ok=True)
    logs_path.mkdir(exist_ok=True)

    return base_path, structures_raw, clean_path, logs_path

def clean_single_structure(input_file: Path, output_file: Path) -> Dict:
    """
    Limpa uma estrutura PDB individual
    """
    pdb_code = input_file.stem

    # Se já foi processada, pula
    if output_file.exists():
        return {
            'pdb_code': pdb_code,
            'status': 'exists',
            'residues': 0
        }

    try:
        parser = PDB.PDBParser(QUIET=True)
        structure = parser.get_structure(pdb_code, str(input_file))

        # Verifica cadeia A
        has_chain_a = any('A' in [c.id for c in m] for m in structure)

        if not has_chain_a:
            return {
                'pdb_code': pdb_code,
                'status': 'no_chain_a',
                'residues': 0
            }

        # Salva apenas cadeia A limpa
        io = PDBIO()
        io.set_structure(structure)
        io.save(str(output_file), ChainAOnlySelect())

        # Conta resíduos
        clean_structure = parser.get_structure(pdb_code, str(output_file))
        residue_count = sum(
            len([r for r in c if r.id[0] == ' '])
            for m in clean_structure for c in m
        )

        if residue_count == 0:
            output_file.unlink()
            return {
                'pdb_code': pdb_code,
                'status': 'empty',
                'residues': 0
            }

        return {
            'pdb_code': pdb_code,
            'status': 'success',
            'residues': residue_count
        }

    except Exception as e:
        return {
            'pdb_code': pdb_code,
            'status': 'error',
            'residues': 0,
            'message': str(e)
        }

def clean_all_structures(raw_dir: Path, clean_dir: Path) -> Dict:
    """
    Processa todas as estruturas em paralelo usando múltiplos processos
    """
    pdb_files = list(raw_dir.glob("*.pdb"))

    if not pdb_files:
        print(f"\nERRO: Nenhum arquivo .pdb encontrado em {raw_dir}")
        return None

    total = len(pdb_files)
    existing_clean = len(list(clean_dir.glob("*.pdb")))

    print(f"\n{'='*60}")
    print(f"LIMPEZA DE ESTRUTURAS PDB - PROCESSAMENTO PARALELO")
    print(f"{'='*60}")
    print(f"\nTotal de estruturas brutas: {total:,}")
    print(f"Já processadas: {existing_clean:,}")
    print(f"Restantes: {total - existing_clean:,}")
    print(f"Workers (CPUs): {MAX_WORKERS}")

    if existing_clean == total:
        print("\n✓ Todas as estruturas já foram limpas!")
        return {
            'total': total,
            'success': existing_clean,
            'exists': existing_clean,
            'no_chain_a': 0,
            'empty': 0,
            'error': 0,
            'total_residues': 0,
            'no_chain_a_codes': [],
            'error_codes': []
        }

    print(f"\nIniciando limpeza paralela...\n")

    results = {
        'total': total,
        'success': 0,
        'exists': 0,
        'no_chain_a': 0,
        'empty': 0,
        'error': 0,
        'total_residues': 0,
        'no_chain_a_codes': [],
        'error_codes': []
    }

    # Processamento paralelo com ThreadPoolExecutor (I/O bound)
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = {
            executor.submit(clean_single_structure, pdb_file, clean_dir / pdb_file.name): pdb_file
            for pdb_file in pdb_files
        }

        with tqdm(total=total, desc="Cleaning structures") as pbar:
            for future in as_completed(futures):
                result = future.result()

                if result['status'] == 'success':
                    results['success'] += 1
                    results['total_residues'] += result['residues']
                elif result['status'] == 'exists':
                    results['exists'] += 1
                elif result['status'] == 'no_chain_a':
                    results['no_chain_a'] += 1
                    results['no_chain_a_codes'].append(result['pdb_code'])
                elif result['status'] == 'empty':
                    results['empty'] += 1
                elif result['status'] == 'error':
                    results['error'] += 1
                    results['error_codes'].append(result['pdb_code'])

                pbar.update(1)
                pbar.set_postfix({
                    'OK': results['success'],
                    'Exist': results['exists'],
                    'NoChainA': results['no_chain_a'],
                    'Error': results['error']
                })

    return results

def save_cleaning_results(logs_path: Path, clean_results: Dict):
    """
    Salva resultados da limpeza
    """
    # Log de estruturas sem cadeia A
    if clean_results['no_chain_a_codes']:
        no_chain_file = logs_path / "no_chain_a.txt"
        with open(no_chain_file, 'w') as f:
            for code in sorted(clean_results['no_chain_a_codes']):
                f.write(f"{code}\n")
        print(f"\nEstruturas sem cadeia A: {no_chain_file}")

    # Log de erros
    if clean_results['error_codes']:
        error_file = logs_path / "cleaning_errors.txt"
        with open(error_file, 'w') as f:
            for code in sorted(clean_results['error_codes']):
                f.write(f"{code}\n")
        print(f"Erros de processamento: {error_file}")

    # Relatório de limpeza
    report_file = logs_path / "cleaning_report.txt"
    with open(report_file, 'w') as f:
        f.write("="*60 + "\n")
        f.write("RELATÓRIO DE LIMPEZA - CATH MAINLY-ALPHA\n")
        f.write("="*60 + "\n\n")
        f.write(f"Total de estruturas: {clean_results['total']:,}\n")
        f.write(f"Limpas com sucesso: {clean_results['success']:,}\n")
        f.write(f"Já existiam: {clean_results['exists']:,}\n")
        f.write(f"Sem cadeia A: {clean_results['no_chain_a']:,}\n")
        f.write(f"Vazias após limpeza: {clean_results['empty']:,}\n")
        f.write(f"Erros: {clean_results['error']:,}\n")
        f.write(f"Total de resíduos: {clean_results['total_residues']:,}\n")

        if clean_results['success'] > 0:
            avg = clean_results['total_residues'] / clean_results['success']
            f.write(f"Média de resíduos/estrutura: {avg:.1f}\n")

        f.write("\n" + "="*60 + "\n")

    print(f"Relatório de limpeza: {report_file}")

def print_cleaning_summary(clean_results: Dict, raw_dir: Path, clean_dir: Path):
    """
    Exibe resumo da limpeza
    """
    print(f"\n{'='*60}")
    print("RESUMO DA LIMPEZA")
    print(f"{'='*60}")
    print(f"\nEstrutura de diretórios:")
    print(f"  Raw (originais):  {raw_dir}")
    print(f"  Clean (limpas):   {clean_dir}")

    print(f"\nResultados da limpeza:")
    print(f"  Total processadas:     {clean_results['total']:,}")
    print(f"  Sucesso:               {clean_results['success']:,}")
    print(f"  Já existiam:           {clean_results['exists']:,}")
    print(f"  Sem cadeia A:          {clean_results['no_chain_a']:,}")
    print(f"  Vazias:                {clean_results['empty']:,}")
    print(f"  Erros:                 {clean_results['error']:,}")

    total_ok = clean_results['success'] + clean_results['exists']
    print(f"\nEstruturas limpas disponíveis: {total_ok:,}")

    if clean_results['total_residues'] > 0:
        print(f"Total de resíduos: {clean_results['total_residues']:,}")

    print(f"{'='*60}\n")

# ========== EXECUÇÃO ==========
print(f"\n{'='*60}")
print("LIMPEZA DE ESTRUTURAS CATH MAINLY-ALPHA")
print("Ambiente Local - Processamento Paralelo")
print(f"{'='*60}\n")

try:
    # 1. Setup de diretórios
    paths = setup_directories()
    if paths[0] is None:
        print("\nExecute primeiro a célula de download!")
    else:
        base_path, raw_dir, clean_dir, logs_path = paths

        print(f"Diretório base: {base_path}")
        print(f"Estruturas raw: {raw_dir}")
        print(f"Estruturas clean: {clean_dir}")

        # 2. Limpa estruturas
        clean_results = clean_all_structures(raw_dir, clean_dir)

        if clean_results is not None:
            # 3. Salva resultados
            save_cleaning_results(logs_path, clean_results)

            # 4. Resumo
            print_cleaning_summary(clean_results, raw_dir, clean_dir)

            print(f"\n✓ Limpeza concluída! Execute a próxima célula para análise de frequência.")

except Exception as e:
    print(f"\nErro crítico na execução: {e}")
    import traceback
    traceback.print_exc()



LIMPEZA DE ESTRUTURAS CATH MAINLY-ALPHA
Ambiente Local - Processamento Paralelo

ERRO: Pasta de estruturas não encontrada em /Users/madsonluna/Documents/cath/structures
Execute primeiro o script de download!

⚠️  Execute primeiro a célula de download!


In [ ]:
#!/usr/bin/env python3
"""
Análise de frequência de aminoácidos nas estruturas limpas
Processamento paralelo otimizado
"""

from pathlib import Path
from collections import Counter
from tqdm.auto import tqdm

from Bio import PDB

# Configurações
BASE_PATH = "/Volumes/promethion/cath-mainly-alpha"

# Aminoácidos padrão
STANDARD_AMINO_ACIDS = {
    'ALA', 'ARG', 'ASN', 'ASP', 'CYS',
    'GLN', 'GLU', 'GLY', 'HIS', 'ILE',
    'LEU', 'LYS', 'MET', 'PHE', 'PRO',
    'SER', 'THR', 'TRP', 'TYR', 'VAL'
}

def analyze_amino_acid_frequency(clean_dir: Path):
    """
    Analisa frequência de aminoácidos em paralelo
    """
    print(f"\n{'='*60}")
    print(f"ANÁLISE DE FREQUÊNCIA DE AMINOÁCIDOS")
    print(f"{'='*60}\n")

    pdb_files = list(clean_dir.glob("*.pdb"))

    if not pdb_files:
        print("ERRO: Nenhuma estrutura limpa encontrada!")
        print(f"Execute primeiro a célula de limpeza!")
        return None, None

    print(f"Analisando {len(pdb_files):,} estruturas...\n")

    global_counter = Counter()
    per_structure = {}
    parser = PDB.PDBParser(QUIET=True)

    with tqdm(total=len(pdb_files), desc="Analyzing") as pbar:
        for pdb_file in pdb_files:
            try:
                structure = parser.get_structure(pdb_file.stem, str(pdb_file))
                local_counter = Counter()

                for model in structure:
                    for chain in model:
                        for residue in chain:
                            if residue.id[0] == ' ':
                                resname = residue.resname.strip()
                                if resname in STANDARD_AMINO_ACIDS:
                                    global_counter[resname] += 1
                                    local_counter[resname] += 1

                per_structure[pdb_file.stem] = dict(local_counter)
            except Exception:
                pass
            pbar.update(1)

    return global_counter, per_structure

def save_frequency_results(analysis_path: Path, global_counter: Counter, per_structure: dict):
    """
    Salva resultados de frequência
    """
    # Frequências globais
    total_residues = sum(global_counter.values())

    freq_file = analysis_path / "amino_acid_frequencies_global.txt"
    with open(freq_file, 'w') as f:
        f.write("="*60 + "\n")
        f.write("FREQUÊNCIA GLOBAL DE AMINOÁCIDOS\n")
        f.write("="*60 + "\n\n")
        f.write(f"Total de resíduos: {total_residues:,}\n")
        f.write(f"Total de estruturas: {len(per_structure):,}\n\n")
        f.write(f"{'AA':<5} {'Contagem':<12} {'Frequência (%)':<15}\n")
        f.write("-"*60 + "\n")

        for aa, count in global_counter.most_common():
            freq = (count / total_residues) * 100
            f.write(f"{aa:<5} {count:<12,} {freq:>14.2f}%\n")

    print(f"\nFrequências globais: {freq_file}")

    # CSV por estrutura
    csv_file = analysis_path / "amino_acid_frequencies_per_structure.csv"
    with open(csv_file, 'w') as f:
        all_aa = sorted(STANDARD_AMINO_ACIDS)
        f.write("PDB_Code," + ",".join(all_aa) + ",Total\n")

        for pdb_code in sorted(per_structure.keys()):
            counts = per_structure[pdb_code]
            row = [pdb_code] + [str(counts.get(aa, 0)) for aa in all_aa]
            row.append(str(sum(counts.values())))
            f.write(",".join(row) + "\n")

    print(f"Frequências por estrutura: {csv_file}")

def print_frequency_summary(global_counter: Counter, per_structure: dict):
    """
    Exibe resumo das análises
    """
    total_residues = sum(global_counter.values())

    print(f"\n{'='*60}")
    print("RESUMO DA ANÁLISE DE FREQUÊNCIA")
    print(f"{'='*60}")
    print(f"\nTotal de estruturas analisadas: {len(per_structure):,}")
    print(f"Total de resíduos: {total_residues:,}")

    print(f"\nTop 5 aminoácidos mais frequentes:")
    for aa, count in global_counter.most_common(5):
        freq = (count / total_residues) * 100
        print(f"  {aa}: {count:,} ({freq:.2f}%)")

    print(f"{'='*60}\n")

# ========== EXECUÇÃO ==========
print(f"\n{'='*60}")
print("ANÁLISE DE FREQUÊNCIA DE AMINOÁCIDOS")
print("Ambiente Local")
print(f"{'='*60}\n")

try:
    base_path = Path(BASE_PATH)
    clean_dir = base_path / "structures_clean"
    analysis_path = base_path / "analysis"

    # Cria pasta de análise se não existir
    analysis_path.mkdir(exist_ok=True)

    # Verifica se há estruturas limpas
    if not clean_dir.exists():
        print(f"\nERRO: Pasta de estruturas limpas não encontrada!")
        print(f"Execute primeiro a célula de limpeza!")
    else:
        # Analisa frequências
        global_counter, per_structure = analyze_amino_acid_frequency(clean_dir)

        if global_counter is not None and per_structure is not None:
            # Salva resultados
            save_frequency_results(analysis_path, global_counter, per_structure)

            # Resumo
            print_frequency_summary(global_counter, per_structure)

except Exception as e:
    print(f"\nErro crítico na execução: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
#!/usr/bin/env python3
"""
Análises avançadas de composição de hélices alpha
Ambiente local com processamento paralelo
"""

from Bio import PDB
from Bio.PDB import DSSP, PPBuilder
from pathlib import Path
from collections import Counter, defaultdict
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import subprocess
import multiprocessing
from concurrent.futures import ProcessPoolExecutor, as_completed

# Configurações para ambiente local
BASE_PATH = "/Volumes/promethion/cath-mainly-alpha"
CLEAN_STRUCTURES_PATH = f"{BASE_PATH}/structures_clean"
MAX_WORKERS = max(1, multiprocessing.cpu_count() - 1)

# Aminoácidos agrupados por propriedades
AA_PROPERTIES = {
    'Hidrofóbico': ['ALA', 'VAL', 'ILE', 'LEU', 'MET', 'PHE', 'TRP', 'PRO'],
    'Polar': ['SER', 'THR', 'CYS', 'TYR', 'ASN', 'GLN'],
    'Carregado positivo': ['LYS', 'ARG', 'HIS'],
    'Carregado negativo': ['ASP', 'GLU'],
    'Especial': ['GLY', 'PRO']
}

# Propensão de aminoácidos para hélices (escala de Chou-Fasman)
HELIX_PROPENSITY = {
    'ALA': 1.42, 'GLU': 1.51, 'LEU': 1.21, 'MET': 1.45,
    'GLN': 1.11, 'LYS': 1.16, 'ARG': 0.98, 'HIS': 1.00,
    'VAL': 1.06, 'ILE': 1.08, 'TYR': 0.69, 'PHE': 1.13,
    'TRP': 1.08, 'THR': 0.83, 'SER': 0.77, 'CYS': 0.70,
    'ASP': 1.01, 'ASN': 0.67, 'GLY': 0.57, 'PRO': 0.57
}

STANDARD_AMINO_ACIDS = {
    'ALA', 'ARG', 'ASN', 'ASP', 'CYS',
    'GLN', 'GLU', 'GLY', 'HIS', 'ILE',
    'LEU', 'LYS', 'MET', 'PHE', 'PRO',
    'SER', 'THR', 'TRP', 'TYR', 'VAL'
}

def install_dssp():
    """
    Instala DSSP no macOS usando Homebrew
    """
    print("\nVerificando instalação do DSSP...")
    
    try:
        # Verifica se DSSP já está instalado
        result = subprocess.run(['which', 'mkdssp'], capture_output=True, text=True)
        if result.returncode == 0:
            print(f"✓ DSSP já instalado em: {result.stdout.strip()}")
            return True
    except Exception:
        pass
    
    print("DSSP não encontrado. Instalando via Homebrew...")
    print("Execute no terminal: brew install brewsci/bio/dssp")
    print("\nOu instale manualmente de: https://swift.cmbi.umcn.nl/gv/dssp/")
    return False

def analyze_single_structure_dssp(pdb_file: Path) -> dict:
    """
    Analisa uma estrutura individual com DSSP
    """
    pdb_code = pdb_file.stem
    parser = PDB.PDBParser(QUIET=True)
    
    try:
        structure = parser.get_structure(pdb_code, str(pdb_file))
        model = structure[0]
        
        # DSSP analysis
        dssp = DSSP(model, str(pdb_file), dssp='mkdssp')
        
        helix_residues = Counter()
        all_residues = Counter()
        helix_positions = defaultdict(lambda: defaultdict(int))
        helix_lengths = []
        
        # Processa estrutura secundária
        ss_sequence = []
        aa_sequence = []
        
        for key in dssp.property_keys:
            residue = dssp[key]
            aa = residue[1]
            ss = residue[2]
            
            ss_sequence.append(ss)
            aa_sequence.append(aa)
            all_residues[aa] += 1
        
        # Identifica hélices
        current_helix = []
        
        for i, (ss, aa) in enumerate(zip(ss_sequence, aa_sequence)):
            if ss in ['H', 'G', 'I']:
                helix_residues[aa] += 1
                current_helix.append(aa)
            else:
                if len(current_helix) >= 4:
                    helix_lengths.append(len(current_helix))
                    
                    for j, res in enumerate(current_helix):
                        if j == 0:
                            helix_positions['N-terminal'][res] += 1
                        elif j == len(current_helix) - 1:
                            helix_positions['C-terminal'][res] += 1
                        else:
                            helix_positions['Middle'][res] += 1
                
                current_helix = []
        
        # Última hélice
        if len(current_helix) >= 4:
            helix_lengths.append(len(current_helix))
            for j, res in enumerate(current_helix):
                if j == 0:
                    helix_positions['N-terminal'][res] += 1
                elif j == len(current_helix) - 1:
                    helix_positions['C-terminal'][res] += 1
                else:
                    helix_positions['Middle'][res] += 1
        
        return {
            'pdb_code': pdb_code,
            'status': 'success',
            'helix_residues': dict(helix_residues),
            'all_residues': dict(all_residues),
            'helix_positions': dict(helix_positions),
            'helix_lengths': helix_lengths
        }
        
    except Exception as e:
        return {
            'pdb_code': pdb_code,
            'status': 'error',
            'message': str(e)
        }

def analyze_secondary_structure_with_dssp(clean_dir: Path, analysis_path: Path):
    """
    Analisa estrutura secundária usando DSSP com processamento paralelo
    """
    print("\nAnalisando estrutura secundária com DSSP...")
    
    if not install_dssp():
        print("\n⚠️  DSSP não instalado. Por favor, instale o DSSP para continuar.")
        return None
    
    pdb_files = list(clean_dir.glob("*.pdb"))
    
    # Contadores globais
    global_helix_residues = Counter()
    global_all_residues = Counter()
    global_helix_positions = defaultdict(lambda: defaultdict(int))
    global_helix_lengths = []
    
    successful = 0
    failed = 0
    
    print(f"Processando {len(pdb_files):,} estruturas com {MAX_WORKERS} workers...\n")
    
    with tqdm(total=len(pdb_files), desc="DSSP Analysis") as pbar:
        for pdb_file in pdb_files:
            result = analyze_single_structure_dssp(pdb_file)
            
            if result['status'] == 'success':
                successful += 1
                
                # Agrega resultados
                for aa, count in result['helix_residues'].items():
                    global_helix_residues[aa] += count
                
                for aa, count in result['all_residues'].items():
                    global_all_residues[aa] += count
                
                for pos_type, pos_data in result['helix_positions'].items():
                    for aa, count in pos_data.items():
                        global_helix_positions[pos_type][aa] += count
                
                global_helix_lengths.extend(result['helix_lengths'])
            else:
                failed += 1
            
            pbar.update(1)
    
    print(f"\nDSSP concluído: {successful} sucessos, {failed} falhas")
    
    return {
        'helix_residues': global_helix_residues,
        'all_residues': global_all_residues,
        'helix_positions': dict(global_helix_positions),
        'helix_lengths': global_helix_lengths
    }

def calculate_helix_propensities(helix_residues: Counter, all_residues: Counter) -> pd.DataFrame:
    """
    Calcula propensões observadas vs esperadas para hélices
    """
    data = []
    
    total_helix = sum(helix_residues.values())
    total_all = sum(all_residues.values())
    
    for aa in sorted(STANDARD_AMINO_ACIDS):
        helix_count = helix_residues.get(aa, 0)
        total_count = all_residues.get(aa, 0)
        
        if total_count > 0:
            freq_helix = (helix_count / total_helix) * 100
            freq_expected = (total_count / total_all) * 100
            propensity_observed = freq_helix / freq_expected if freq_expected > 0 else 0
            propensity_theoretical = HELIX_PROPENSITY.get(aa, 1.0)
            
            data.append({
                'AA': aa,
                'Count_Helix': helix_count,
                'Count_Total': total_count,
                'Freq_Helix': freq_helix,
                'Freq_Total': freq_expected,
                'Propensity_Observed': propensity_observed,
                'Propensity_Theoretical': propensity_theoretical
            })
    
    df = pd.DataFrame(data)
    return df

def analyze_helix_positions(helix_positions: dict) -> pd.DataFrame:
    """
    Analisa preferências de aminoácidos em diferentes posições da hélice
    """
    data = []
    
    positions = ['N-terminal', 'Middle', 'C-terminal']
    
    for pos in positions:
        pos_counts = helix_positions.get(pos, {})
        total = sum(pos_counts.values())
        
        for aa in sorted(STANDARD_AMINO_ACIDS):
            count = pos_counts.get(aa, 0)
            freq = (count / total * 100) if total > 0 else 0
            
            data.append({
                'Position': pos,
                'AA': aa,
                'Count': count,
                'Frequency': freq
            })
    
    df = pd.DataFrame(data)
    return df

def analyze_hydrophobic_patterns(clean_dir: Path) -> dict:
    """
    Analisa padrões de hidrofobicidade em hélices
    """
    print("\nAnalisando padrões de hidrofobicidade...")
    
    hydrophobic = set(['ALA', 'VAL', 'ILE', 'LEU', 'MET', 'PHE', 'TRP'])
    
    pdb_files = list(clean_dir.glob("*.pdb"))
    parser = PDB.PDBParser(QUIET=True)
    
    heptad_positions = defaultdict(lambda: {'hydrophobic': 0, 'total': 0})
    
    with tqdm(total=len(pdb_files), desc="Hydrophobic patterns") as pbar:
        for pdb_file in pdb_files:
            try:
                structure = parser.get_structure(pdb_file.stem, str(pdb_file))
                
                for model in structure:
                    for chain in model:
                        residues = [r for r in chain if r.id[0] == ' ']
                        
                        for i, residue in enumerate(residues):
                            pos_in_heptad = i % 7
                            resname = residue.resname.strip()
                            
                            heptad_positions[pos_in_heptad]['total'] += 1
                            if resname in hydrophobic:
                                heptad_positions[pos_in_heptad]['hydrophobic'] += 1
            
            except Exception:
                pass
            
            pbar.update(1)
    
    return dict(heptad_positions)

def analyze_aa_properties_distribution(all_residues: Counter) -> pd.DataFrame:
    """
    Analisa distribuição por propriedades físico-químicas
    """
    data = []
    
    total = sum(all_residues.values())
    
    for property_name, aa_list in AA_PROPERTIES.items():
        count = sum(all_residues.get(aa, 0) for aa in aa_list)
        freq = (count / total * 100) if total > 0 else 0
        
        data.append({
            'Property': property_name,
            'Count': count,
            'Frequency': freq
        })
    
    df = pd.DataFrame(data)
    return df

def plot_helix_propensities(df: pd.DataFrame, analysis_path: Path):
    """
    Gráfico de propensões de hélices
    """
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    x = np.arange(len(df))
    width = 0.35
    
    axes[0].bar(x - width/2, df['Propensity_Observed'], width,
                label='Propensão Observada', color='steelblue', edgecolor='black')
    axes[0].bar(x + width/2, df['Propensity_Theoretical'], width,
                label='Propensão Teórica (Chou-Fasman)', color='coral', edgecolor='black')
    axes[0].axhline(y=1.0, color='red', linestyle='--', alpha=0.5, label='Propensão neutra')
    axes[0].set_xlabel('Aminoácido')
    axes[0].set_ylabel('Propensão para Hélice')
    axes[0].set_title('Propensão de Aminoácidos para Hélices Alpha')
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(df['AA'], rotation=0)
    axes[0].legend()
    axes[0].grid(axis='y', alpha=0.3)
    
    axes[1].scatter(df['Freq_Total'], df['Freq_Helix'], s=100, alpha=0.6, color='steelblue')
    for idx, row in df.iterrows():
        axes[1].annotate(row['AA'], (row['Freq_Total'], row['Freq_Helix']),
                        fontsize=9, ha='center')
    
    max_val = max(df['Freq_Total'].max(), df['Freq_Helix'].max())
    axes[1].plot([0, max_val], [0, max_val], 'r--', alpha=0.5, label='Frequência igual')
    
    axes[1].set_xlabel('Frequência Total (%)')
    axes[1].set_ylabel('Frequência em Hélices (%)')
    axes[1].set_title('Frequência de Aminoácidos: Total vs Hélices')
    axes[1].legend()
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    
    plot_file = analysis_path / "helix_propensities.png"
    plt.savefig(plot_file, dpi=300, bbox_inches='tight')
    print(f"Gráfico salvo em: {plot_file}")
    plt.close()

def plot_helix_positions(df: pd.DataFrame, analysis_path: Path):
    """
    Heatmap de preferências por posição na hélice
    """
    pivot = df.pivot(index='AA', columns='Position', values='Frequency')
    pivot = pivot[['N-terminal', 'Middle', 'C-terminal']]
    
    plt.figure(figsize=(8, 10))
    sns.heatmap(pivot, annot=True, fmt='.1f', cmap='YlOrRd',
                cbar_kws={'label': 'Frequência (%)'})
    plt.title('Preferência de Aminoácidos por Posição na Hélice')
    plt.ylabel('Aminoácido')
    plt.xlabel('Posição')
    plt.tight_layout()
    
    plot_file = analysis_path / "helix_positions.png"
    plt.savefig(plot_file, dpi=300, bbox_inches='tight')
    print(f"Gráfico salvo em: {plot_file}")
    plt.close()

def plot_helix_length_distribution(helix_lengths: list, analysis_path: Path):
    """
    Distribuição de comprimentos de hélices
    """
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.hist(helix_lengths, bins=50, color='steelblue', edgecolor='black', alpha=0.7)
    plt.xlabel('Comprimento da Hélice (resíduos)')
    plt.ylabel('Frequência')
    plt.title('Distribuição de Comprimentos de Hélices')
    plt.grid(axis='y', alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.boxplot(helix_lengths, vert=True)
    plt.ylabel('Comprimento da Hélice (resíduos)')
    plt.title('Box Plot - Comprimento de Hélices')
    plt.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    
    plot_file = analysis_path / "helix_lengths.png"
    plt.savefig(plot_file, dpi=300, bbox_inches='tight')
    print(f"Gráfico salvo em: {plot_file}")
    plt.close()
    
    print(f"\nEstatísticas de comprimento de hélices:")
    print(f"  Média: {np.mean(helix_lengths):.1f} resíduos")
    print(f"  Mediana: {np.median(helix_lengths):.1f} resíduos")
    print(f"  Desvio padrão: {np.std(helix_lengths):.1f}")
    print(f"  Min: {min(helix_lengths)}, Max: {max(helix_lengths)}")

def plot_hydrophobic_heptad(heptad_data: dict, analysis_path: Path):
    """
    Gráfico de padrão heptad
    """
    positions = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
    hydrophobic_freq = []
    
    for i in range(7):
        data = heptad_data[i]
        freq = (data['hydrophobic'] / data['total'] * 100) if data['total'] > 0 else 0
        hydrophobic_freq.append(freq)
    
    plt.figure(figsize=(10, 6))
    plt.bar(positions, hydrophobic_freq, color='steelblue', edgecolor='black')
    plt.axhline(y=np.mean(hydrophobic_freq), color='red', linestyle='--',
                alpha=0.5, label=f'Média: {np.mean(hydrophobic_freq):.1f}%')
    plt.xlabel('Posição no Heptad Repeat')
    plt.ylabel('Frequência de Aminoácidos Hidrofóbicos (%)')
    plt.title('Padrão Heptad: Distribuição de Resíduos Hidrofóbicos')
    plt.legend()
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    
    plot_file = analysis_path / "heptad_pattern.png"
    plt.savefig(plot_file, dpi=300, bbox_inches='tight')
    print(f"Gráfico salvo em: {plot_file}")
    plt.close()

def save_comprehensive_report(analysis_path: Path, propensity_df: pd.DataFrame,
                              position_df: pd.DataFrame, properties_df: pd.DataFrame,
                              helix_lengths: list):
    """
    Salva relatório abrangente
    """
    report_file = analysis_path / "helix_analysis_comprehensive_report.txt"
    
    with open(report_file, 'w') as f:
        f.write("="*80 + "\n")
        f.write("ANÁLISE ABRANGENTE DE COMPOSIÇÃO DE HÉLICES ALPHA\n")
        f.write("Estruturas CATH Mainly-Alpha\n")
        f.write("="*80 + "\n\n")
        
        f.write("### PROPENSÃO DE AMINOÁCIDOS PARA HÉLICES ###\n\n")
        f.write(f"{'AA':<5} {'Helix':<10} {'Total':<10} {'Prop.Obs':<12} {'Prop.Teor':<12}\n")
        f.write("-"*80 + "\n")
        for _, row in propensity_df.iterrows():
            f.write(f"{row['AA']:<5} {row['Count_Helix']:<10} {row['Count_Total']:<10} "
                   f"{row['Propensity_Observed']:<12.2f} {row['Propensity_Theoretical']:<12.2f}\n")
        
        f.write("\n### TOP 5 FORMADORES DE HÉLICES (Propensão Observada) ###\n")
        top_formers = propensity_df.nlargest(5, 'Propensity_Observed')
        for idx, row in top_formers.iterrows():
            f.write(f"  {idx+1}. {row['AA']}: {row['Propensity_Observed']:.2f}\n")
        
        f.write("\n### DISTRIBUIÇÃO POR PROPRIEDADES ###\n\n")
        for _, row in properties_df.iterrows():
            f.write(f"{row['Property']:<25}: {row['Frequency']:>6.2f}% ({row['Count']:,})\n")
        
        f.write("\n### ESTATÍSTICAS DE COMPRIMENTO DE HÉLICES ###\n\n")
        f.write(f"  Número total de hélices: {len(helix_lengths):,}\n")
        f.write(f"  Comprimento médio: {np.mean(helix_lengths):.1f} resíduos\n")
        f.write(f"  Mediana: {np.median(helix_lengths):.1f} resíduos\n")
        f.write(f"  Desvio padrão: {np.std(helix_lengths):.1f}\n")
        f.write(f"  Intervalo: {min(helix_lengths)} - {max(helix_lengths)} resíduos\n")
        
        percentiles = [25, 50, 75, 90, 95]
        f.write("\n  Percentis:\n")
        for p in percentiles:
            val = np.percentile(helix_lengths, p)
            f.write(f"    {p}%: {val:.1f} resíduos\n")
        
        f.write("\n" + "="*80 + "\n")
    
    print(f"\nRelatório abrangente salvo em: {report_file}")
    
    propensity_df.to_csv(analysis_path / "helix_propensities.csv", index=False)
    position_df.to_csv(analysis_path / "helix_positions.csv", index=False)
    properties_df.to_csv(analysis_path / "aa_properties_distribution.csv", index=False)
    
    print("Arquivos CSV salvos:")
    print("  - helix_propensities.csv")
    print("  - helix_positions.csv")
    print("  - aa_properties_distribution.csv")

def run_helix_analysis():
    """
    Executa todas as análises de hélices
    """
    clean_dir = Path(CLEAN_STRUCTURES_PATH)
    analysis_path = Path(BASE_PATH) / "analysis"
    analysis_path.mkdir(exist_ok=True)
    
    print("="*80)
    print("ANÁLISE AVANÇADA DE HÉLICES ALPHA")
    print("Ambiente Local - Processamento Paralelo")
    print("="*80)
    
    # 1. Análise com DSSP
    dssp_results = analyze_secondary_structure_with_dssp(clean_dir, analysis_path)
    
    if dssp_results is None:
        return
    
    # 2. Calcula propensões
    propensity_df = calculate_helix_propensities(
        dssp_results['helix_residues'],
        dssp_results['all_residues']
    )
    
    # 3. Analisa posições
    position_df = analyze_helix_positions(dssp_results['helix_positions'])
    
    # 4. Analisa propriedades
    properties_df = analyze_aa_properties_distribution(dssp_results['all_residues'])
    
    # 5. Analisa padrões hidrofóbicos
    heptad_data = analyze_hydrophobic_patterns(clean_dir)
    
    # 6. Gera gráficos
    print("\nGerando visualizações...")
    plot_helix_propensities(propensity_df, analysis_path)
    plot_helix_positions(position_df, analysis_path)
    plot_helix_length_distribution(dssp_results['helix_lengths'], analysis_path)
    plot_hydrophobic_heptad(heptad_data, analysis_path)
    
    # 7. Salva relatórios
    save_comprehensive_report(
        analysis_path,
        propensity_df,
        position_df,
        properties_df,
        dssp_results['helix_lengths']
    )
    
    print("\n" + "="*80)
    print("ANÁLISE CONCLUÍDA")
    print("="*80)

# Executar análise
if __name__ == "__main__":
    run_helix_analysis()


In [ ]:
#!/usr/bin/env python3
"""
Análise de tipos de hélices e composição específica por tipo
Ambiente local com processamento paralelo
"""

from Bio import PDB
from Bio.PDB import DSSP
from pathlib import Path
from collections import Counter, defaultdict
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import multiprocessing

# Configurações para ambiente local
BASE_PATH = "/Volumes/promethion/cath-mainly-alpha"
CLEAN_STRUCTURES_PATH = f"{BASE_PATH}/structures_clean"
MAX_WORKERS = max(1, multiprocessing.cpu_count() - 1)

# Definição de tipos de hélices (DSSP)
HELIX_TYPES = {
    'H': 'Alpha helix',
    'G': '3-10 helix',
    'I': 'Pi helix'
}

# Características estruturais dos tipos de hélices
HELIX_CHARACTERISTICS = {
    'H': {
        'name': 'Alpha helix',
        'residues_per_turn': 3.6,
        'rise_per_residue': 1.5,
        'hydrogen_bond': 'i to i+4',
        'typical_length': '10-20 residues'
    },
    'G': {
        'name': '3-10 helix',
        'residues_per_turn': 3.0,
        'rise_per_residue': 2.0,
        'hydrogen_bond': 'i to i+3',
        'typical_length': '3-5 residues'
    },
    'I': {
        'name': 'Pi helix',
        'residues_per_turn': 4.4,
        'rise_per_residue': 1.15,
        'hydrogen_bond': 'i to i+5',
        'typical_length': '7-10 residues'
    }
}

STANDARD_AMINO_ACIDS = {
    'ALA', 'ARG', 'ASN', 'ASP', 'CYS',
    'GLN', 'GLU', 'GLY', 'HIS', 'ILE',
    'LEU', 'LYS', 'MET', 'PHE', 'PRO',
    'SER', 'THR', 'TRP', 'TYR', 'VAL'
}

def classify_helix_types(clean_dir: Path) -> dict:
    """
    Classifica e analisa diferentes tipos de hélices
    """
    print("\nClassificando tipos de hélices com DSSP...")
    
    pdb_files = list(clean_dir.glob("*.pdb"))
    parser = PDB.PDBParser(QUIET=True)
    
    helix_type_counts = Counter()
    helix_type_residues = defaultdict(Counter)
    helix_type_lengths = defaultdict(list)
    helix_type_positions = defaultdict(lambda: defaultdict(Counter))
    transitions = defaultdict(Counter)
    
    successful = 0
    failed = 0
    structures_with_helices = defaultdict(int)
    
    with tqdm(total=len(pdb_files), desc="Classifying helix types") as pbar:
        for pdb_file in pdb_files:
            pdb_code = pdb_file.stem
            
            try:
                structure = parser.get_structure(pdb_code, str(pdb_file))
                model = structure[0]
                
                dssp = DSSP(model, str(pdb_file), dssp='mkdssp')
                
                ss_sequence = []
                aa_sequence = []
                
                for key in dssp.property_keys:
                    residue = dssp[key]
                    aa = residue[1]
                    ss = residue[2]
                    
                    ss_sequence.append(ss)
                    aa_sequence.append(aa)
                
                current_helix = {'type': None, 'residues': [], 'start': 0}
                
                for i, (ss, aa) in enumerate(zip(ss_sequence, aa_sequence)):
                    if ss in ['H', 'G', 'I']:
                        if current_helix['type'] == ss:
                            current_helix['residues'].append(aa)
                        else:
                            if current_helix['type'] and len(current_helix['residues']) >= 3:
                                helix_type = current_helix['type']
                                residues = current_helix['residues']
                                
                                helix_type_counts[helix_type] += 1
                                helix_type_lengths[helix_type].append(len(residues))
                                structures_with_helices[helix_type] += 1
                                
                                for j, res in enumerate(residues):
                                    helix_type_residues[helix_type][res] += 1
                                    
                                    if j == 0:
                                        helix_type_positions[helix_type]['N-terminal'][res] += 1
                                    elif j == len(residues) - 1:
                                        helix_type_positions[helix_type]['C-terminal'][res] += 1
                                    else:
                                        helix_type_positions[helix_type]['Middle'][res] += 1
                            
                            if current_helix['type'] and current_helix['type'] != ss:
                                transitions[current_helix['type']][ss] += 1
                            
                            current_helix = {'type': ss, 'residues': [aa], 'start': i}
                    else:
                        if current_helix['type'] and len(current_helix['residues']) >= 3:
                            helix_type = current_helix['type']
                            residues = current_helix['residues']
                            
                            helix_type_counts[helix_type] += 1
                            helix_type_lengths[helix_type].append(len(residues))
                            
                            for j, res in enumerate(residues):
                                helix_type_residues[helix_type][res] += 1
                                
                                if j == 0:
                                    helix_type_positions[helix_type]['N-terminal'][res] += 1
                                elif j == len(residues) - 1:
                                    helix_type_positions[helix_type]['C-terminal'][res] += 1
                                else:
                                    helix_type_positions[helix_type]['Middle'][res] += 1
                        
                        current_helix = {'type': None, 'residues': [], 'start': 0}
                
                if current_helix['type'] and len(current_helix['residues']) >= 3:
                    helix_type = current_helix['type']
                    residues = current_helix['residues']
                    
                    helix_type_counts[helix_type] += 1
                    helix_type_lengths[helix_type].append(len(residues))
                    
                    for j, res in enumerate(residues):
                        helix_type_residues[helix_type][res] += 1
                        
                        if j == 0:
                            helix_type_positions[helix_type]['N-terminal'][res] += 1
                        elif j == len(residues) - 1:
                            helix_type_positions[helix_type]['C-terminal'][res] += 1
                        else:
                            helix_type_positions[helix_type]['Middle'][res] += 1
                
                successful += 1
                
            except Exception as e:
                failed += 1
            
            pbar.update(1)
    
    print(f"\nClassificação concluída: {successful} sucessos, {failed} falhas")
    
    return {
        'helix_counts': dict(helix_type_counts),
        'helix_residues': dict(helix_type_residues),
        'helix_lengths': dict(helix_type_lengths),
        'helix_positions': dict(helix_type_positions),
        'transitions': dict(transitions),
        'structures_analyzed': successful
    }

def analyze_helix_type_composition(helix_residues: dict) -> pd.DataFrame:
    """
    Analisa composição de aminoácidos por tipo de hélice
    """
    data = []
    
    for helix_type, residue_counts in helix_residues.items():
        total = sum(residue_counts.values())
        helix_name = HELIX_TYPES.get(helix_type, helix_type)
        
        for aa in sorted(STANDARD_AMINO_ACIDS):
            count = residue_counts.get(aa, 0)
            freq = (count / total * 100) if total > 0 else 0
            
            data.append({
                'Helix_Type': helix_type,
                'Helix_Name': helix_name,
                'AA': aa,
                'Count': count,
                'Frequency': freq
            })
    
    df = pd.DataFrame(data)
    return df

def compare_helix_types(helix_residues: dict) -> pd.DataFrame:
    """
    Compara os aminoácidos mais frequentes entre tipos de hélices
    """
    data = []
    
    for helix_type, residue_counts in helix_residues.items():
        total = sum(residue_counts.values())
        helix_name = HELIX_TYPES.get(helix_type, helix_type)
        
        top_aa = residue_counts.most_common(10)
        
        for rank, (aa, count) in enumerate(top_aa, 1):
            freq = (count / total * 100) if total > 0 else 0
            
            data.append({
                'Helix_Type': helix_type,
                'Helix_Name': helix_name,
                'Rank': rank,
                'AA': aa,
                'Count': count,
                'Frequency': freq
            })
    
    df = pd.DataFrame(data)
    return df

def statistical_comparison(composition_df: pd.DataFrame) -> pd.DataFrame:
    """
    Teste estatístico para diferenças significativas entre tipos
    """
    helix_types = composition_df['Helix_Type'].unique()
    
    if len(helix_types) < 2:
        return pd.DataFrame()
    
    results = []
    
    for aa in sorted(STANDARD_AMINO_ACIDS):
        aa_data = composition_df[composition_df['AA'] == aa]
        
        if 'H' in helix_types and 'G' in helix_types:
            alpha_freq = aa_data[aa_data['Helix_Type'] == 'H']['Frequency'].values
            g310_freq = aa_data[aa_data['Helix_Type'] == 'G']['Frequency'].values
            
            if len(alpha_freq) > 0 and len(g310_freq) > 0:
                diff = alpha_freq[0] - g310_freq[0]
                
                results.append({
                    'AA': aa,
                    'Alpha_Freq': alpha_freq[0],
                    '3-10_Freq': g310_freq[0],
                    'Difference': diff,
                    'Enriched_In': 'Alpha' if diff > 0 else '3-10'
                })
    
    df = pd.DataFrame(results)
    df = df.sort_values('Difference', ascending=False, key=abs)
    
    return df

def plot_helix_type_distribution(helix_counts: dict, analysis_path: Path):
    """
    Gráfico de distribuição de tipos de hélices
    """
    types = []
    counts = []
    names = []
    
    for helix_type in ['H', 'G', 'I']:
        if helix_type in helix_counts:
            types.append(helix_type)
            counts.append(helix_counts[helix_type])
            names.append(HELIX_TYPES[helix_type])
    
    total = sum(counts)
    percentages = [(c/total*100) for c in counts]
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    colors = ['steelblue', 'coral', 'lightgreen']
    axes[0].bar(names, counts, color=colors[:len(names)], edgecolor='black')
    axes[0].set_ylabel('Número de Hélices')
    axes[0].set_title('Distribuição de Tipos de Hélices')
    axes[0].grid(axis='y', alpha=0.3)
    
    for i, (name, count, pct) in enumerate(zip(names, counts, percentages)):
        axes[0].text(i, count, f'{count:,}\n({pct:.1f}%)',
                    ha='center', va='bottom', fontweight='bold')
    
    axes[1].pie(counts, labels=names, autopct='%1.1f%%', colors=colors[:len(names)],
                startangle=90, wedgeprops={'edgecolor': 'black'})
    axes[1].set_title('Proporção de Tipos de Hélices')
    
    plt.tight_layout()
    
    plot_file = analysis_path / "helix_type_distribution.png"
    plt.savefig(plot_file, dpi=300, bbox_inches='tight')
    print(f"Gráfico salvo em: {plot_file}")
    plt.close()

def plot_helix_type_composition(composition_df: pd.DataFrame, analysis_path: Path):
    """
    Heatmap comparativo de composição por tipo de hélice
    """
    pivot = composition_df.pivot(index='AA', columns='Helix_Name', values='Frequency')
    
    if 'Alpha helix' in pivot.columns:
        pivot = pivot.sort_values('Alpha helix', ascending=False)
    
    plt.figure(figsize=(10, 12))
    sns.heatmap(pivot, annot=True, fmt='.1f', cmap='YlOrRd',
                cbar_kws={'label': 'Frequência (%)'}, linewidths=0.5)
    plt.title('Composição de Aminoácidos por Tipo de Hélice', fontsize=14, fontweight='bold')
    plt.ylabel('Aminoácido', fontsize=12)
    plt.xlabel('Tipo de Hélice', fontsize=12)
    plt.tight_layout()
    
    plot_file = analysis_path / "helix_type_composition_heatmap.png"
    plt.savefig(plot_file, dpi=300, bbox_inches='tight')
    print(f"Gráfico salvo em: {plot_file}")
    plt.close()

def plot_top_amino_acids_by_type(comparison_df: pd.DataFrame, analysis_path: Path):
    """
    Gráfico dos aminoácidos mais frequentes por tipo
    """
    helix_types = comparison_df['Helix_Name'].unique()
    n_types = len(helix_types)
    
    fig, axes = plt.subplots(1, n_types, figsize=(6*n_types, 6))
    
    if n_types == 1:
        axes = [axes]
    
    colors = ['steelblue', 'coral', 'lightgreen']
    
    for idx, helix_name in enumerate(helix_types):
        data = comparison_df[comparison_df['Helix_Name'] == helix_name]
        top10 = data.nsmallest(10, 'Rank')
        
        axes[idx].barh(top10['AA'], top10['Frequency'], color=colors[idx], edgecolor='black')
        axes[idx].set_xlabel('Frequência (%)')
        axes[idx].set_title(f'{helix_name}\nTop 10 Aminoácidos', fontweight='bold')
        axes[idx].invert_yaxis()
        axes[idx].grid(axis='x', alpha=0.3)
        
        for i, (aa, freq) in enumerate(zip(top10['AA'], top10['Frequency'])):
            axes[idx].text(freq, i, f' {freq:.1f}%', va='center')
    
    plt.tight_layout()
    
    plot_file = analysis_path / "helix_type_top_amino_acids.png"
    plt.savefig(plot_file, dpi=300, bbox_inches='tight')
    print(f"Gráfico salvo em: {plot_file}")
    plt.close()

def plot_helix_length_comparison(helix_lengths: dict, analysis_path: Path):
    """
    Comparação de distribuição de comprimentos por tipo
    """
    fig, axes = plt.subplots(2, 1, figsize=(12, 10))
    
    types_to_plot = [t for t in ['H', 'G', 'I'] if t in helix_lengths]
    colors = ['steelblue', 'coral', 'lightgreen']
    
    data_to_plot = [helix_lengths[t] for t in types_to_plot]
    labels = [HELIX_TYPES[t] for t in types_to_plot]
    
    bp = axes[0].boxplot(data_to_plot, labels=labels, patch_artist=True)
    for patch, color in zip(bp['boxes'], colors[:len(types_to_plot)]):
        patch.set_facecolor(color)
    
    axes[0].set_ylabel('Comprimento (resíduos)')
    axes[0].set_title('Distribuição de Comprimentos por Tipo de Hélice')
    axes[0].grid(axis='y', alpha=0.3)
    
    for helix_type, color in zip(types_to_plot, colors[:len(types_to_plot)]):
        lengths = helix_lengths[helix_type]
        axes[1].hist(lengths, bins=30, alpha=0.5, label=HELIX_TYPES[helix_type],
                    color=color, edgecolor='black')
    
    axes[1].set_xlabel('Comprimento (resíduos)')
    axes[1].set_ylabel('Frequência')
    axes[1].set_title('Histograma de Comprimentos')
    axes[1].legend()
    axes[1].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    
    plot_file = analysis_path / "helix_length_by_type.png"
    plt.savefig(plot_file, dpi=300, bbox_inches='tight')
    print(f"Gráfico salvo em: {plot_file}")
    plt.close()

def plot_statistical_differences(stat_df: pd.DataFrame, analysis_path: Path):
    """
    Gráfico de diferenças estatísticas entre Alpha e 3-10
    """
    if stat_df.empty:
        return
    
    top_diff = stat_df.head(10)
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    colors = ['steelblue' if x > 0 else 'coral' for x in top_diff['Difference']]
    axes[0].barh(top_diff['AA'], top_diff['Difference'], color=colors, edgecolor='black')
    axes[0].axvline(x=0, color='black', linestyle='-', linewidth=0.8)
    axes[0].set_xlabel('Diferença de Frequência (%)')
    axes[0].set_title('Diferenças Mais Significativas:\nAlpha helix vs 3-10 helix', fontweight='bold')
    axes[0].invert_yaxis()
    axes[0].grid(axis='x', alpha=0.3)
    
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='steelblue', label='Enriquecido em Alpha'),
        Patch(facecolor='coral', label='Enriquecido em 3-10')
    ]
    axes[0].legend(handles=legend_elements)
    
    axes[1].scatter(top_diff['Alpha_Freq'], top_diff['3-10_Freq'],
                   s=200, alpha=0.6, c=colors, edgecolor='black', linewidth=2)
    
    for _, row in top_diff.iterrows():
        axes[1].annotate(row['AA'],
                        (row['Alpha_Freq'], row['3-10_Freq']),
                        fontsize=10, ha='center', fontweight='bold')
    
    max_val = max(top_diff['Alpha_Freq'].max(), top_diff['3-10_Freq'].max())
    axes[1].plot([0, max_val], [0, max_val], 'k--', alpha=0.3, label='Frequência igual')
    
    axes[1].set_xlabel('Frequência em Alpha helix (%)')
    axes[1].set_ylabel('Frequência em 3-10 helix (%)')
    axes[1].set_title('Comparação Direta: Alpha vs 3-10', fontweight='bold')
    axes[1].legend()
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    
    plot_file = analysis_path / "helix_type_statistical_comparison.png"
    plt.savefig(plot_file, dpi=300, bbox_inches='tight')
    print(f"Gráfico salvo em: {plot_file}")
    plt.close()

def save_helix_type_report(analysis_path: Path, results: dict,
                           composition_df: pd.DataFrame, comparison_df: pd.DataFrame,
                           stat_df: pd.DataFrame):
    """
    Salva relatório detalhado sobre tipos de hélices
    """
    report_file = analysis_path / "helix_types_comprehensive_report.txt"
    
    with open(report_file, 'w') as f:
        f.write("="*80 + "\n")
        f.write("ANÁLISE DETALHADA DE TIPOS DE HÉLICES\n")
        f.write("Estruturas CATH Mainly-Alpha\n")
        f.write("="*80 + "\n\n")
        
        f.write(f"Estruturas analisadas: {results['structures_analyzed']}\n\n")
        
        f.write("### DISTRIBUIÇÃO DE TIPOS DE HÉLICES ###\n\n")
        total_helices = sum(results['helix_counts'].values())
        f.write(f"{'Tipo':<20} {'Nome':<20} {'Contagem':<12} {'Percentual':<12}\n")
        f.write("-"*80 + "\n")
        
        for helix_type in ['H', 'G', 'I']:
            if helix_type in results['helix_counts']:
                count = results['helix_counts'][helix_type]
                pct = (count / total_helices * 100) if total_helices > 0 else 0
                name = HELIX_TYPES[helix_type]
                f.write(f"{helix_type:<20} {name:<20} {count:<12,} {pct:>10.1f}%\n")
        
        f.write("\n### CARACTERÍSTICAS ESTRUTURAIS ###\n\n")
        for helix_type in ['H', 'G', 'I']:
            if helix_type in results['helix_counts']:
                char = HELIX_CHARACTERISTICS[helix_type]
                f.write(f"{char['name']}:\n")
                f.write(f"  Resíduos por volta: {char['residues_per_turn']}\n")
                f.write(f"  Aumento por resíduo: {char['rise_per_residue']} Å\n")
                f.write(f"  Ligação de hidrogênio: {char['hydrogen_bond']}\n")
                f.write(f"  Comprimento típico: {char['typical_length']}\n\n")
        
        f.write("### ESTATÍSTICAS DE COMPRIMENTO ###\n\n")
        for helix_type in ['H', 'G', 'I']:
            if helix_type in results['helix_lengths']:
                lengths = results['helix_lengths'][helix_type]
                f.write(f"{HELIX_TYPES[helix_type]}:\n")
                f.write(f"  Média: {np.mean(lengths):.1f} resíduos\n")
                f.write(f"  Mediana: {np.median(lengths):.1f} resíduos\n")
                f.write(f"  Desvio padrão: {np.std(lengths):.1f}\n")
                f.write(f"  Min-Max: {min(lengths)} - {max(lengths)} resíduos\n\n")
        
        f.write("### TOP 5 AMINOÁCIDOS POR TIPO DE HÉLICE ###\n\n")
        for helix_type in ['H', 'G', 'I']:
            if helix_type in results['helix_residues']:
                f.write(f"{HELIX_TYPES[helix_type]}:\n")
                residues = results['helix_residues'][helix_type]
                total = sum(residues.values())
                
                for rank, (aa, count) in enumerate(residues.most_common(5), 1):
                    freq = (count / total * 100) if total > 0 else 0
                    f.write(f"  {rank}. {aa}: {freq:.2f}% ({count:,} resíduos)\n")
                f.write("\n")
        
        if not stat_df.empty:
            f.write("### DIFERENÇAS SIGNIFICATIVAS (Alpha vs 3-10) ###\n\n")
            f.write("Top 10 aminoácidos com maiores diferenças:\n\n")
            f.write(f"{'AA':<5} {'Alpha (%)':<12} {'3-10 (%)':<12} {'Diferença':<12} {'Enriquecido':<15}\n")
            f.write("-"*80 + "\n")
            
            for _, row in stat_df.head(10).iterrows():
                f.write(f"{row['AA']:<5} {row['Alpha_Freq']:<12.2f} "
                       f"{row['3-10_Freq']:<12.2f} {row['Difference']:<12.2f} "
                       f"{row['Enriched_In']:<15}\n")
        
        f.write("\n" + "="*80 + "\n")
    
    print(f"\nRelatório completo salvo em: {report_file}")
    
    composition_df.to_csv(analysis_path / "helix_type_composition.csv", index=False)
    comparison_df.to_csv(analysis_path / "helix_type_top_residues.csv", index=False)
    
    if not stat_df.empty:
        stat_df.to_csv(analysis_path / "helix_type_statistical_comparison.csv", index=False)
    
    print("Arquivos CSV salvos:")
    print("  - helix_type_composition.csv")
    print("  - helix_type_top_residues.csv")
    print("  - helix_type_statistical_comparison.csv")

def run_helix_type_analysis():
    """
    Executa análise completa de tipos de hélices
    """
    clean_dir = Path(CLEAN_STRUCTURES_PATH)
    analysis_path = Path(BASE_PATH) / "analysis"
    analysis_path.mkdir(exist_ok=True)
    
    print("="*80)
    print("ANÁLISE DE TIPOS DE HÉLICES E COMPOSIÇÃO")
    print("Ambiente Local - Processamento Paralelo")
    print("="*80)
    
    results = classify_helix_types(clean_dir)
    composition_df = analyze_helix_type_composition(results['helix_residues'])
    comparison_df = compare_helix_types(results['helix_residues'])
    stat_df = statistical_comparison(composition_df)
    
    print("\nGerando visualizações...")
    plot_helix_type_distribution(results['helix_counts'], analysis_path)
    plot_helix_type_composition(composition_df, analysis_path)
    plot_top_amino_acids_by_type(comparison_df, analysis_path)
    plot_helix_length_comparison(results['helix_lengths'], analysis_path)
    
    if not stat_df.empty:
        plot_statistical_differences(stat_df, analysis_path)
    
    save_helix_type_report(analysis_path, results, composition_df, comparison_df, stat_df)
    
    print("\n" + "="*80)
    print("RESUMO DA ANÁLISE")
    print("="*80)
    
    for helix_type in ['H', 'G', 'I']:
        if helix_type in results['helix_counts']:
            count = results['helix_counts'][helix_type]
            name = HELIX_TYPES[helix_type]
            residues = results['helix_residues'][helix_type]
            top_aa = residues.most_common(1)[0] if residues else ('N/A', 0)
            
            print(f"\n{name}:")
            print(f"  Total: {count:,} hélices")
            print(f"  Aminoácido mais frequente: {top_aa[0]} ({top_aa[1]:,} ocorrências)")
            
            if helix_type in results['helix_lengths']:
                lengths = results['helix_lengths'][helix_type]
                print(f"  Comprimento médio: {np.mean(lengths):.1f} resíduos")
    
    print("\n" + "="*80)
    print("ANÁLISE CONCLUÍDA")
    print("="*80)

if __name__ == "__main__":
    run_helix_type_analysis()
